In [1]:
import pandas as pd
import numpy as np
import os, sys

idx = pd.IndexSlice

In [2]:
# %load /home/jonathan/.ipython/profile_default/startup/01-setup.py
# start up settings for jupyter notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys

# use plt.style.available() to check out available styles
plt.style.use('seaborn-white')
plt.rcParams['font.size'] = 20
plt.rcParams['axes.labelsize'] = 15.0
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['legend.fontsize'] = 20

%matplotlib inline

# set the max column width
pd.options.display.max_colwidth = 1000

# to avoid have warnings from chained assignments
pd.options.mode.chained_assignment = None


fp = '../../prediction-data/clst-2018-12-generic_50-inc0-net1-predictive-output.csv'
df = pd.read_csv(fp, header=[0, 1])
df.rename(columns={
    'Unnamed: 0_level_1': '',
    'Unnamed: 1_level_1': '',
    'Unnamed: 2_level_1': ''
}, level=1, inplace=True)

columns = list(df.columns)
decomposition_type = 'generic_50'
df['decomposition'] = decomposition_type
columns = [('model', ''), ('log', ''), ('decomposition', ''), ('SP label', '')] + columns[3:]
df = df[columns]

df.to_csv(fp, index=False)

In [6]:
data_dir = '../../prediction-data/'

df_list = []

for f in os.listdir(data_dir):
    if 'inc0' in f or 'final-data' in f:
        continue
    fp = os.path.join(data_dir, f)
    
    if not os.path.isfile(fp):
        continue

    df = pd.read_csv(fp, header=[0, 1])
    df.rename(columns={
        'Unnamed: 0_level_1': '',
        'Unnamed: 1_level_1': '',
        'Unnamed: 2_level_1': '',
        'Unnamed: 3_level_1': ''
    }, level=1, inplace=True)
    df_list.append(df)
    


In [7]:
base_df = pd.concat(df_list, axis=0, sort=False)
base_df.set_index(['model', 'log', 'decomposition', 'SP label'], inplace=True)

# remove traces for which all algorithms are invalid
astar_invalid = (base_df.loc[:,('is_valid', 'astar')] == False) | (base_df.loc[:, ('is_valid', 'astar')].isnull())
inc3_invalid = (base_df.loc[:,('is_valid', 'inc3')] == False) | (base_df.loc[:,('is_valid', 'inc3')].isnull())
recomp_astar_invalid = (base_df.loc[:,('is_valid', 'recomp-astar')] == False) | (base_df.loc[:,('is_valid', 'recomp-astar')].isnull())
recomp_inc3_invalid = (base_df.loc[:,('is_valid', 'recomp-inc3')] == False) | (base_df.loc[:,('is_valid', 'recomp-inc3')].isnull())
all_invalid = astar_invalid & inc3_invalid & recomp_astar_invalid & recomp_inc3_invalid

before_n_rows = base_df.shape[0]
print('Before filtering all invalid no. of rows: {}'.format(before_n_rows))

base_df = base_df.loc[~all_invalid, :]

after_n_rows = base_df.shape[0]
print('After filtering all invalid no. of rows: {}'.format(after_n_rows))
print('{} traces invalid for all algorithms'.format(before_n_rows - after_n_rows))

cost_is_neg_recomp_astar = base_df.loc[:, ('Cost of the alignment', 'recomp-astar')] == -1
cost_is_neg_recomp_inc3 = base_df.loc[:, ('Cost of the alignment', 'recomp-inc3')] == -1

before_n_rows = base_df.shape[0]
print('Before filtering negative cost rows: {}'.format(before_n_rows))

base_df = base_df.loc[~(cost_is_neg_recomp_astar | cost_is_neg_recomp_inc3), :]

after_n_rows = base_df.shape[0]
print('After filtering negative cost rows: {}'.format(after_n_rows))
print('{} traces invalid'.format(before_n_rows - after_n_rows))

Before filtering all invalid no. of rows: 406130
After filtering all invalid no. of rows: 405795
335 traces invalid for all algorithms
Before filtering negative cost rows: 405795
After filtering negative cost rows: 405357
438 traces invalid


In [8]:
base_df.columns.get_level_values(level=0).unique()

Index(['Cost of the alignment', 'Heuristics computed', 'Heuristics derived',
       'Heuristics estimated', 'Markings added to closed set',
       'Markings polled from queue', 'Markings queued', 'Markings reached',
       'Min', 'Time to compute alignment (us)',
       'Total Time including setup (us)', 'is_valid', 'n_invis_move',
       'n_log_move', 'n_model_move', 'n_sync_move', 'result_dir'],
      dtype='object')

### Find and correct errors with algorithms yielding invalid alignments but has the minimum time

#### Total Time including setup (us)

In [11]:
min_total_astar = base_df.loc[:,('Min', 'Total Time including setup (us)')] == 'astar'
min_total_inc3 = base_df.loc[:,('Min', 'Total Time including setup (us)')] == 'inc3'
min_total_r_astar = base_df.loc[:,('Min', 'Total Time including setup (us)')] == 'recomp-astar'
min_total_r_inc3 = base_df.loc[:,('Min', 'Total Time including setup (us)')] == 'recomp-inc3'

false_min_astar = base_df.loc[astar_invalid & min_total_astar, ]
false_min_inc3 = base_df.loc[inc3_invalid & min_total_inc3, ]
false_min_r_astar = base_df.loc[recomp_astar_invalid & min_total_r_astar, ]
false_min_r_inc3 = base_df.loc[recomp_inc3_invalid & min_total_r_inc3, ]

print('No. of false min astar: {}'.format(false_min_astar.shape[0]))
print('No. of false min inc3: {}'.format(false_min_inc3.shape[0]))
print('No. of false min recomp astar: {}'.format(false_min_r_astar.shape[0]))
print('No. of false min recomp inc3: {}'.format(false_min_r_inc3.shape[0]))

No. of false min astar: 0
No. of false min inc3: 0
No. of false min recomp astar: 0
No. of false min recomp inc3: 0


#### Time to compute alignment (us)

In [12]:
col = ('Min', 'Time to compute alignment (us)')
min_align_astar = base_df.loc[:,col] == 'astar'
min_align_inc3 = base_df.loc[:,col] == 'inc3'
min_align_r_astar = base_df.loc[:,col] == 'recomp-astar'
min_align_r_inc3 = base_df.loc[:,col] == 'recomp-inc3'

false_min_astar = base_df.loc[astar_invalid & min_align_astar, ]
false_min_inc3 = base_df.loc[inc3_invalid & min_align_inc3, ]
false_min_r_astar = base_df.loc[recomp_astar_invalid & min_align_r_astar, ]
false_min_r_inc3 = base_df.loc[recomp_inc3_invalid & min_align_r_inc3, ]

print('No. of false min align time astar: {}'.format(false_min_astar.shape[0]))
print('No. of false min align time inc3: {}'.format(false_min_inc3.shape[0]))
print('No. of false min align time recomp astar: {}'.format(false_min_r_astar.shape[0]))
print('No. of false min align time recomp inc3: {}'.format(false_min_r_inc3.shape[0]))

No. of false min align time astar: 0
No. of false min align time inc3: 0
No. of false min align time recomp astar: 0
No. of false min align time recomp inc3: 0


In [13]:
base_df.describe()

Cost of the alignment                                               \
                      astar           inc3   recomp-astar    recomp-inc3   
count         405357.000000  360414.000000  405357.000000  225605.000000   
mean              57.354786      56.540306      57.446039      53.332923   
std              137.512584     135.766728     138.066793     130.484134   
min                0.000000       0.000000       0.000000       0.000000   
25%                0.000000       0.000000       0.000000       0.000000   
50%                0.000000       0.000000       0.000000       0.000000   
75%               22.000000      24.000000      22.000000      14.000000   
max             1256.000000    1256.000000    3090.000000    1224.000000   

      Heuristics computed                                               \
                    astar           inc3   recomp-astar    recomp-inc3   
count       405357.000000  360414.000000  405357.000000  225605.000000   
mean           694.780088       6.639609     765.382446      79.806853   
std           2037.921552      32.927709    2090.003476      44.027746   
min              1.000000       1.000000      12.000000      12.000000   
25%              1.000000       1.000000      71.000000      69.000000   
50%              1.000000       1.000000      74.000000      75.000000   
75%              8.000000       2.000000      87.000000      78.000000   
max          38800.000000    1191.000000   25251.000000     989.000000   

      Heuristics derived                     ...           n_log_move  \
                   astar           inc3      ...         recomp-astar   
count      405357.000000  360414.000000      ...        405357.000000   
mean          304.113174     264.569032      ...             4.248865   
std           645.611416     696.914106      ...            10.451754   
min             3.000000       3.000000      ...             0.000000   
25%            58.000000      58.000000      ...             0.000000   
50%            82.000000      82.000000      ...             0.000000   
75%           174.000000     166.000000      ...             1.000000   
max         23185.000000   16646.000000      ...           309.000000   

                       n_model_move                                \
         recomp-inc3          astar           inc3   recomp-astar   
count  225605.000000  405357.000000  360414.000000  405357.000000   
mean        3.938986       3.724714       3.658046       3.739348   
std         9.903699       8.551612       8.359467       8.596439   
min         0.000000      -1.000000      -1.000000       0.000000   
25%         0.000000       0.000000       0.000000       0.000000   
50%         0.000000       0.000000       0.000000       0.000000   
75%         1.000000       2.000000       2.000000       2.000000   
max        98.000000      81.000000      78.000000      81.000000   

                        n_sync_move                                \
         recomp-inc3          astar           inc3   recomp-astar   
count  225605.000000  405357.000000  360414.000000  405357.000000   
mean        3.485765      67.201094      67.147600      67.267073   
std         8.066557      35.104791      35.594206      35.226600   
min         0.000000      -1.000000      -1.000000       0.000000   
25%         0.000000      45.000000      45.000000      45.000000   
50%         0.000000      58.000000      57.000000      58.000000   
75%         2.000000      79.000000      79.000000      79.000000   
max        78.000000     419.000000     419.000000     419.000000   

                      
         recomp-inc3  
count  225605.000000  
mean       66.868208  
std        36.636203  
min         0.000000  
25%        45.000000  
50%        57.000000  
75%        79.000000  
max       419.000000  

[8 rows x 56 columns]

## Model and trace features

In [14]:
fp = '../../prediction-data/features/net1-feature.csv'
df = pd.read_csv(fp)

fp = '../../prediction-data/features/P241-feature.csv'
df = pd.read_csv(fp)

In [15]:
df.head()

,caseid,trace_length,n_activity,activity_repeat_mean,activity_repeat_std,snp_n_transition,snp_n_inv_transition,snp_n_dup_transition,snp_n_uniq_transition,snp_inv_transition_in_degree_mean,...,subnet_n_dup_transition_mean,subnet_n_dup_transition_std,subnet_n_uniq_transition_mean,subnet_n_uniq_transition_std,subnet_n_place_mean,subnet_n_place_std,subnet_n_arc_mean,subnet_n_arc_std,decomposition,log
0,case_0,106.0,93.0,1.139785,0.543567,355.0,26.0,40.0,289.0,2.0,...,0.0,0.0,40.0,41.581246,52.0,67.557383,116.666667,141.454351,sese_manual,L50
1,case_1,94.0,91.0,1.032967,0.179540,331.0,26.0,12.0,293.0,2.0,...,0.0,0.0,40.0,41.581246,52.0,67.557383,116.666667,141.454351,sese_manual,L50
2,case_10,112.0,90.0,1.244444,0.768935,367.0,26.0,62.0,279.0,2.0,...,0.0,0.0,40.0,41.581246,52.0,67.557383,116.666667,141.454351,sese_manual,L50
3,case_100,105.0,91.0,1.153846,0.613244,353.0,26.0,42.0,285.0,2.0,...,0.0,0.0,40.0,41.581246,52.0,67.557383,116.666667,141.454351,sese_manual,L50
4,case_101,88.0,87.0,1.011494,0.107211,319.0,26.0,4.0,289.0,2.0,...,0.0,0.0,40.0,41.581246,52.0,67.557383,116.666667,141.454351,sese_manual,L50


In [16]:
feature_dir = '../../prediction-data/features/'

feature_df_list = []

for f in os.listdir(feature_dir):
    fp = os.path.join(feature_dir, f)
    model = f.replace('-feature.csv', '')
    df = pd.read_csv(fp)
    df['model'] = model
    feature_df_list.append(df)
    

In [17]:
feature_df = pd.concat(feature_df_list, axis=0, sort=False)
feature_df.rename(columns={'caseid': 'SP label'}, inplace=True) # for later convenience on merging with base_df
feature_df.set_index(['model', 'log', 'decomposition', 'SP label'], inplace=True)
feature_df.columns = pd.MultiIndex.from_product([['model_trace_features'], feature_df.columns])

In [18]:
full_df = base_df.merge(feature_df, on=['model', 'log', 'decomposition', 'SP label'], how='inner')
out_fp = '../../prediction-data/final-data.csv'

full_df.reset_index(drop=False).to_csv(out_fp, index=False)

### Getting the no-duplicates dataframe
Two rows are considered duplicates if they have the values in:
- all of the model_trace_features columns
- same cost of the alignment

Note that they do not need the same alignment, i.e., same number of legal move types

In [21]:
print('Number of rows before filtering duplicates: {}'.format(full_df.shape[0]))

select_level_0 = ('model_trace_features', 'Cost of the alignment')
select_level_1 = slice(None)

duplicate_rows = full_df.loc[:, idx[select_level_0, select_level_1]].duplicated()
uniq_full_df = full_df[~duplicate_rows]

print('Number of rows after filtering duplicates: {}'.format(uniq_full_df.shape[0]))
print('Number of rows removed: {}'.format(full_df.shape[0] - uniq_full_df.shape[0]))

out_fp = '../../prediction-data/uniq-final-data.csv'
uniq_full_df.reset_index(drop=False).to_csv(out_fp, index=False)

Number of rows before filtering duplicates: 251637
Number of rows after filtering duplicates: 191221
Number of rows removed: 60416


In [ ]:
full_df_description = full_df.describe()

In [ ]:
col = ('Min', 'Total Time including setup (us)')
min_total_astar = full_df.loc[:, col] == 'astar'
min_total_inc3 = full_df.loc[:, col] == 'inc3'
min_total_r_astar = full_df.loc[:, col] == 'recomp-astar'
min_total_r_inc3 = full_df.loc[:, col] == 'recomp-inc3'

total_win_astar_secs = full_df.loc[min_total_astar, idx[('Total Time including setup (us)'), :]] / 1000000
total_win_inc3_secs = full_df.loc[min_total_inc3, idx[('Total Time including setup (us)'), :]] / 1000000
total_win_r_astar_secs = full_df.loc[min_total_r_astar, idx[('Total Time including setup (us)'), :]] / 1000000
total_win_r_inc3_secs = full_df.loc[min_total_r_inc3, idx[('Total Time including setup (us)'), :]] / 1000000

In [ ]:
total_win_r_inc3_secs.describe()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

class_dist = full_df.loc[:, idx[('Min'), ('Total Time including setup (us)')]].value_counts()
class_dist.plot(kind='bar', ax=ax, color='dodgerblue', alpha=0.5, edgecolor='black', linewidth=1.5);

xticklabels = [
    'CLASSIC', 'CLASSIC-SP', 'RECOMPOSE', 'RECOMPOSE-SP'
]
ax.set_xticklabels(xticklabels, size=20, rotation=0)

total_wins = [
    total_win_astar_secs,
    total_win_inc3_secs,
    total_win_r_astar_secs,
    total_win_r_inc3_secs
]

algo = [
    'astar', 'inc3', 'recomp-astar', 'recomp-inc3'
]

means = [total_wins[i].loc[:, ('Total Time including setup (us)', algo[i])].mean() for i in range(4)]
std = [ss.std() for ss in total_wins]

# ax_twin = ax.twinx()

# ax_twin.plot(means, marker='^', linestyle='', )

fig.tight_layout()
fig.savefig('./class-dist.svg', bbox_inches='tight', rasterized=True)

# Scrap... 

In [ ]:
concat_df.head(2)

### Class distribution

In [ ]:
concat_df.loc[(concat_df[('Min', 'Total Time including setup (us)')] == 'astar'),('Cost of the alignment','astar')].describe()

In [ ]:
concat_df.loc[(concat_df[('Min', 'Total Time including setup (us)')] == 'inc3'),('Cost of the alignment', 'inc3')].describe()

In [ ]:
temp_df.describe()

In [ ]:
temp_df = concat_df['n_sync_move'] + concat_df['n_invis_move'] + concat_df['n_model_move'] + concat_df['n_log_move']
temp_df.columns = pd.MultiIndex.from_product([['alignment_length'], ['astar', 'inc3', 'recomp-astar', 'recomp-inc3']])
concat_df = pd.concat([concat_df, temp_df], axis=1)

In [ ]:
idx = pd.IndexSlice

astar_invalid = concat_df.loc[:,('is_valid','astar')] == False
min_is_astar = concat_df.loc[:,('Min', 'Total Time including setup (us)')] == 'astar'
concat_df.loc[(astar_invalid)&(min_is_astar),idx[('is_valid', slice(None))]]

In [ ]:
concat_df.columns.get_level_values(0)

In [ ]:
idx = pd.IndexSlice
concat_df.loc[:, idx[('Min', 'is_valid'),('Total Time including setup (us)', 'astar', 'inc3', 'recomp-astar', 'recomp-inc3')]]

In [ ]:
astar_invalid = concat_df.loc[:,('is_valid', 'astar')] == False
inc3_invalid = concat_df.loc[:,('is_valid', 'inc3')] == False
recomp_astar_invalid = concat_df.loc[:,('is_valid', 'recomp-astar')] == False
recomp_inc3_invalid = concat_df.loc[:,('is_valid', 'recomp-inc3')] == False
min_total_astar = concat_df.loc[:,('Min', 'Total Time including setup (us)')] == 'astar'
min_total_inc3 = concat_df.loc[:,('Min', 'Total Time including setup (us)')] == 'inc3'
min_total_recomp_astar = concat_df.loc[:,('Min', 'Total Time including setup (us)')] == 'recomp-astar'
min_total_recomp_inc3 = concat_df.loc[:,('Min', 'Total Time including setup (us)')] == 'recomp-inc3'
min_align_astar = concat_df.loc[:,('Min', 'Time to compute alignment (us)')] == 'astar'
min_align_inc3 = concat_df.loc[:,('Min', 'Time to compute alignment (us)')] == 'inc3'
min_align_recomp_astar = concat_df.loc[:,('Min', 'Time to compute alignment (us)')] == 'recomp-astar'
min_align_recomp_inc3 = concat_df.loc[:,('Min', 'Time to compute alignment (us)')] == 'recomp-inc3'

concat_df.loc[astar_invalid & inc3_invalid & recomp_astar_invalid & recomp_inc3_invalid,:].shape

In [ ]:
concat_df.loc[astar_invalid | inc3_invalid | recomp_astar_invalid | recomp_inc3_invalid,:].shape

In [ ]:
concat_df.loc[astar_invalid & min_astar, idx[('Min', 'is_valid'),('Total Time including setup (us)', 'astar')]]

In [ ]:
concat_df.loc[astar_invalid & min_align_astar,:].shape

In [ ]:
concat_df.loc[inc3_invalid & min_total_inc3,:].shape

In [ ]:
concat_df.loc[recomp_astar_invalid & min_total_recomp_astar, :].shape

In [ ]:
concat_df.loc[recomp_inc3_invalid & min_total_recomp_inc3, :].shape

In [ ]:
concat_df['Total Time including setup (us)'].max()

In [ ]:
concat_df.loc[(concat_df[('Min', 'Total Time including setup (us)')] == 'recomp-inc3'),[('Cost of the alignment','astar'), ('')]].describe()